In [2]:
# import libraries and functions
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt

import ee
import geemap 

# import json
from datetime import datetime
import os

In [3]:
# autheniticate
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Ab_5qlkao17g3wgXTHSIHOslF-a5ooUOMZ2z4IeFzs6zLnB-jAEx8oDYFUs



Successfully saved authorization token.


In [ ]:


# Load shapefile using geopandas
shapefile_path = "rg_glaciers.shp"
gdf = gpd.read_file(shapefile_path)

# Convert to Earth Engine FeatureCollection
fc = geemap.geopandas_to_ee(gdf)

# Define months of interest
months = [6, 7, 9, 10]  # June, July, September, October

# Sentinel-2 image collection and date range
s2_collection = "COPERNICUS/S2_SR"  # Surface Reflectance
start_date = '2015-06-23'  # Start of Sentinel-2A
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Function to calculate NDSI
def add_ndsi(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    return img.addBands(ndsi)

# Dictionary to hold results
monthly_results = {}

# Loop through months and extract mean NDSI per polygon
for month in months:
    # Filter by month and valid dates
    collection = (
        ee.ImageCollection(s2_collection)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(month, month, 'month'))
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
        .map(add_ndsi)
        .select('NDSI')
    )

    # Calculate mean NDSI for the month
    mean_image = collection.mean()

    # Reduce over regions (polygons)
    reduced = mean_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    # Get results as a list
    reduced_list = reduced.getInfo()

    # Store in dictionary
    monthly_results[month] = reduced_list

# Print sample output
for month, data in monthly_results.items():
    print(f"\nMonth: {month}")
    for feature in data[:3]:  # print first 3 polygons as sample
        print(f"ID: {feature['properties'].get('id', 'n/a')}, Mean NDSI: {feature['properties'].get('mean')}")
